In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from scipy.misc import imread, imresize
from PIL import Image
import pandas as pd
from pathlib import Path

# from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention, CrossGPTLayer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

VERSION = '10'
WORD_NUM = '6'

from score import evalscores

import json

curr_path = Path(os.getcwd())

all_reports_df = pd.read_pickle(os.path.join(str(curr_path.parent), 'NLMCXR_data/all_reports_tags_df.pkl'))

NLMCXR_path = os.path.join(str(curr_path.parent), 'NLMCXR_data')

def caption_image_beam_search(encoder, decoder, image_path, word_map, beam_size=3):
    """
    Reads an image and captions it with beam search.

    :param encoder: encoder model
    :param decoder: decoder model
    :param image_path: path to image
    :param word_map: word map
    :param beam_size: number of sequences to consider at each decode-step
    :return: caption, weights for visualization
    """
    # print(image_path)

    k = beam_size
    vocab_size = len(word_map)

    # Read image and process
    img = imread(image_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    img = imresize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    encoder_out = encoder(image)  # (1, enc_image_size, enc_image_size, encoder_dim)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    # Flatten encoding
    encoder_out = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
    num_pixels = encoder_out.size(1)

    # We'll treat the problem as having a batch size of k
    encoder_out = encoder_out.expand(k, num_pixels, encoder_dim)  # (k, num_pixels, encoder_dim)

    # Tensor to store top k previous words at each step; now they're just <start>
    k_prev_words = torch.LongTensor([[word_map['<start>']]] * k).to(device)  # (k, 1)

    # Tensor to store top k sequences; now they're just <start>
    seqs = k_prev_words  # (k, 1)

    # Tensor to store top k sequences' scores; now they're just 0
    top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)

    # Tensor to store top k sequences' alphas; now they're just 1s
    seqs_alpha = torch.ones(k, 1, enc_image_size, enc_image_size).to(device)  # (k, 1, enc_image_size, enc_image_size)

    # Lists to store completed sequences, their alphas and scores
    complete_seqs = list()
    complete_seqs_alpha = list()
    complete_seqs_scores = list()

    # Start decoding
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)

    # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
    while True:
        

        embeddings = decoder.embedding(k_prev_words).squeeze(1)  # (s, embed_dim)

        awe, alpha = decoder.attention(encoder_out, h)  # (s, encoder_dim), (s, num_pixels)

        alpha = alpha.view(-1, enc_image_size, enc_image_size)  # (s, enc_image_size, enc_image_size)

        gate = decoder.sigmoid(decoder.f_beta(h))  # gating scalar, (s, encoder_dim)
        awe = gate * awe

        h, c = decoder.decode_step(torch.cat([embeddings, awe], dim=1), (h, c))  # (s, decoder_dim)

        scores = decoder.fc(h)  # (s, vocab_size)
        
#         if step == 4:
#             print(seqs, scores)
        
        scores = F.log_softmax(scores, dim=1)

        # Add
        scores = top_k_scores.expand_as(scores) + scores

        # For the first step, all k points will have the same scores (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
        else:
            # Unroll and find top scores, and their unrolled indices
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)

        # Convert unrolled indices to actual indices of scores
        prev_word_inds = top_k_words / vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)

        # Add new words to sequences, alphas
        seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)
        seqs_alpha = torch.cat([seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)],
                               dim=1)  # (s, step+1, enc_image_size, enc_image_size)

        # Which sequences are incomplete (didn't reach <end>)?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                           next_word != word_map['<end>']]
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))

        # Set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist())
            complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        # Proceed with incomplete sequences
        if k == 0:
            break
        seqs = seqs[incomplete_inds]
        seqs_alpha = seqs_alpha[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)
        
        # Break if things have been going on too long
        if step > 50:
            break
        step += 1

    i = complete_seqs_scores.index(max(complete_seqs_scores))
    seq = complete_seqs[i]
    alphas = complete_seqs_alpha[i]

    return seq, alphas

def visualize_att(image_path, seq, alphas, rev_word_map, smooth=True, visualize=True):
    """
    Visualizes caption with weights at every word.

    Adapted from paper authors' repo: https://github.com/kelvinxu/arctic-captions/blob/master/alpha_visualization.ipynb

    :param image_path: path to image that has been captioned
    :param seq: caption
    :param alphas: weights
    :param rev_word_map: reverse word mapping, i.e. ix2word
    :param smooth: smooth weights?
    """
    image = Image.open(image_path)
    image = image.resize([14 * 24, 14 * 24], Image.LANCZOS)

    words = [rev_word_map[ind] for ind in seq]
    
    if visualize:
        for t in range(len(words)):
            if t > 50:
                break
            plt.subplot(np.ceil(len(words) / 5.), 5, t + 1)

            plt.text(0, 1, '%s' % (words[t]), color='black', backgroundcolor='white', fontsize=12)
            plt.imshow(image)
            current_alpha = alphas[t, :]
            if smooth:
                alpha = skimage.transform.pyramid_expand(current_alpha.numpy(), upscale=24, sigma=8)
            else:
                alpha = skimage.transform.resize(current_alpha.numpy(), [14 * 24, 14 * 24])
            if t == 0:
                plt.imshow(alpha, alpha=0)
            else:
                plt.imshow(alpha, alpha=0.8)
            plt.set_cmap(cm.Greys_r)
            plt.axis('off')
        plt.show()
    
    return words


def get_print(img_name, visualize=True, print_res=True):
    parser = argparse.ArgumentParser(description='Show, Attend, and Tell - Tutorial - Generate Caption')
    parser.add_argument('--img', '-i', default=f'{NLMCXR_path}/images/{img_name}.png', help='path to image')
    parser.add_argument('--model', '-m', default=f'BEST_checkpoint_NLMCXR_v{VERSION}_1_cap_per_img_{WORD_NUM}_min_word_freq.pth.tar', help='path to model')
    parser.add_argument('--word_map', '-wm', default=f'{NLMCXR_path}/WORDMAP_NLMCXR_v{VERSION}_1_cap_per_img_{WORD_NUM}_min_word_freq.json',  help='path to word map JSON')
    parser.add_argument('--beam_size', '-b', default=5, type=int, help='beam size for beam search')
    parser.add_argument('--dont_smooth', dest='smooth', action='store_false', help='do not smooth alpha overlay')
    
    args = args = parser.parse_args('')
    
    checkpoint = torch.load(args.model)
    decoder = checkpoint['decoder']
    decoder = decoder.to(device)
    decoder.eval()
    encoder = checkpoint['encoder']
    encoder = encoder.to(device)
    encoder.eval()

    # Load word map (word2ix)
    with open(args.word_map, 'r') as j:
        word_map = json.load(j)
    rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

    # Encode, decode with attention and beam search
    seq, alphas = caption_image_beam_search(encoder, decoder, args.img, word_map, args.beam_size)
    alphas = torch.FloatTensor(alphas)
    
    if print_res:
        print(asd[ asd['images'] == f'{img_name}']['processed_findings'].iloc[0])

    # Visualize caption and attention of best sequence
    words = visualize_att(args.img, seq, alphas, rev_word_map, args.smooth, visualize=visualize)
    
    return words

asd = pd.read_pickle(f'{NLMCXR_path}/all_reports_df.pkl')

all_reports_df = pd.read_pickle(f'{NLMCXR_path}/all_reports_tags_df.pkl')
    # all_reports_df = all_reports_df[ all_reports_df['processed_findings'].notnull() ]
# train_df, val_df, test_df = train_validate_test_split(all_reports_df, .9, .05, seed=42)
train_df = all_reports_df[:-600]
val_df = all_reports_df[-600:-300]
# test_df = all_reports_df[-300:]
test_df = all_reports_df[-300:]

test_predicts = []
test_images = test_df.images.values

from tqdm import tqdm

for img_name in tqdm(test_images):
    words = get_print(img_name, visualize=False, print_res=False)
    test_predicts.append(words)
    
max_len = 100

test_image_captions = [[sents.split()][:max_len] for sents in test_df['preprocessed_full_report']]

test_predicts = [words[1:-1] for words in test_predicts]

evalscores(test_predicts, test_image_captions)

I0605 19:35:54.171853 140296825583424 file_utils.py:41] PyTorch version 1.2.0 available.
/home/dchesakov/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dchesakov/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dchesakov/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

Bleu_1: 0.164
Bleu_2: 0.100
Bleu_3: 0.069
Bleu_4: 0.049
ROUGE_L: 0.256
CIDEr: 0.169


In [2]:
# test_predicts = [[ i if i != ',' else '.' for i in pred] for pred in test_predicts]

## GPT-2 generation

In [2]:
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup, get_constant_schedule, get_cosine_schedule_with_warmup,
                                  BertConfig, BertForMaskedLM, BertTokenizer,
                                  GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
                                  OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                                  RobertaConfig, RobertaForMaskedLM, RobertaTokenizer,
                                  DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)

from transformers import GPT2Tokenizer, GPT2Model

import os
import numpy as np

import torch

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(f'{NLMCXR_path}/output3')

I0605 19:41:07.617610 140296825583424 tokenization_utils.py:895] Model name '/home/dchesakov/skoltech_image_cap/NLMCXR_data/output3' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming '/home/dchesakov/skoltech_image_cap/NLMCXR_data/output3' is a path, a model identifier, or url to a directory containing tokenizer files.
I0605 19:41:07.624328 140296825583424 tokenization_utils.py:924] Didn't find file /home/dchesakov/skoltech_image_cap/NLMCXR_data/output3/added_tokens.json. We won't load it.
I0605 19:41:07.629217 140296825583424 tokenization_utils.py:977] loading file /home/dchesakov/skoltech_image_cap/NLMCXR_data/output3/vocab.json
I0605 19:41:07.631656 140296825583424 tokenization_utils.py:977] loading file /home/dchesakov/skoltech_image_cap/NLMCXR_data/output3/merges.txt
I0605 19:41:07.634448 140296825583424 tokenization_utils.py:977] loading file None
I0605 19:41:07.636971 140296825583424 tokenization_utils.py:977] loading file /home

In [4]:
gpt_model = GPT2LMHeadModel.from_pretrained(f'{NLMCXR_path}/output3', pad_token_id=tokenizer.eos_token_id)

I0605 19:41:08.237813 140296825583424 configuration_utils.py:284] loading configuration file /home/dchesakov/skoltech_image_cap/NLMCXR_data/output3/config.json
I0605 19:41:08.238732 140296825583424 configuration_utils.py:322] Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "vocab_size": 50257
}

I0605 19:41:08.239582 140296825583424 modeling_utils.py:610] loading weights file /home/dchesakov/skoltech_image_cap/NLMCXR_data/output3/pytorch_model.bin


In [5]:
pred_sents = [' '.join(i) for i in test_predicts]

In [6]:
pred_sents

['no acute cardiopulmonary abnormality . the heart is normal in size . the mediastinum is unremarkable .',
 'no acute disease . the heart is normal in size . the mediastinum is unremarkable . the lungs are clear .',
 'no acute cardiopulmonary abnormality . the heart is normal in size . the mediastinum is unremarkable . the lungs are clear .',
 'no acute cardiopulmonary disease . the heart is normal in size . the mediastinum is unremarkable . the lungs are clear .',
 'no acute disease . the heart is normal in size . the mediastinum is unremarkable . the lungs are clear .',
 'no acute cardiopulmonary abnormality . the lungs are clear and expanded . heart and mediastinum normal .',
 'no acute cardiopulmonary abnormality . the heart is normal in size . the mediastinum is unremarkable . the lungs are clear .',
 'no acute disease . the heart is normal in size . the mediastinum is unremarkable . the lungs are clear .',
 'no acute disease . the heart is normal in size . the mediastinum is unre

In [7]:
gpt_preds = []

In [8]:
pred_sents[0]

'no acute cardiopulmonary abnormality . the heart is normal in size . the mediastinum is unremarkable .'

In [9]:
for sent in tqdm(pred_sents):
    input_ids = torch.tensor(tokenizer.encode(sent, add_special_tokens=True)).unsqueeze(0)
    beam_output = gpt_model.generate(
                                input_ids, 
                                max_length=200, 
                                num_beams=5, 
                                early_stopping=True
                            )
    gpt_preds.append(tokenizer.decode(beam_output[0]).lower().replace('..', '.').replace('.', ' . '))

100%|██████████| 300/300 [04:34<00:00,  1.12it/s]


In [10]:
gpt_predicts = [i.split()[:] for i in gpt_preds]

In [12]:
evalscores(gpt_predicts, test_image_captions)

Bleu_1: 0.181
Bleu_2: 0.110
Bleu_3: 0.075
Bleu_4: 0.054
ROUGE_L: 0.258
CIDEr: 0.180
